In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import dateutil.parser

pd.set_option ('display.max_columns', None)
pd.set_option ('display.max_rows', None)
weather_data1 = pd.read_csv('Weather_Data/OBS_AWS_TIM_202201-02.csv', encoding = 'euc_kr')
weather_data2 = pd.read_csv('Weather_Data/OBS_AWS_TIM_202203-04.csv', encoding = 'euc_kr')
weather_data3 = pd.read_csv('Weather_Data/OBS_AWS_TIM_202205-06.csv', encoding = 'euc_kr')

weather_data4 = pd.read_csv('Weather_Data/OBS_AWS_TIM_20210101_20210331.csv', encoding = 'euc_kr')
weather_data5 = pd.read_csv('Weather_Data/OBS_AWS_TIM_20210401_20210630.csv', encoding = 'euc_kr')
weather_data6 = pd.read_csv('Weather_Data/OBS_AWS_TIM_20210701_20210930.csv', encoding = 'euc_kr')
weather_data7 = pd.read_csv('Weather_Data/OBS_AWS_TIM_20211001_20212031.csv', encoding = 'euc_kr')

In [2]:
vars_to_drop = ['풍향(deg)', '해면기압(hPa)']

weather_data1 = weather_data1.drop(vars_to_drop, axis = 1)
weather_data2 = weather_data2.drop(vars_to_drop, axis = 1)
weather_data3 = weather_data3.drop(vars_to_drop, axis = 1)

weather_data4 = weather_data4.drop(vars_to_drop, axis = 1)
weather_data5 = weather_data5.drop(vars_to_drop, axis = 1)
weather_data6 = weather_data6.drop(vars_to_drop, axis = 1)
weather_data7 = weather_data7.drop(vars_to_drop, axis = 1)

In [3]:
weather_data = weather_data1.append([weather_data2, weather_data3,weather_data4, weather_data5, weather_data6, weather_data7])

/tmp/ipykernel_44706/1096615632.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weather_data = weather_data1.append([weather_data2, weather_data3,weather_data4, weather_data5, weather_data6, weather_data7])


In [4]:
print(weather_data1.shape)
print(weather_data2.shape)
print(weather_data3.shape)
print(weather_data4.shape)
print(weather_data5.shape)
print(weather_data6.shape)
print(weather_data7.shape)
print(weather_data.shape)

(742082, 8)
(768602, 8)
(768252, 8)
(1121784, 8)
(1134024, 8)
(1145881, 8)
(1151005, 8)
(6831630, 8)


In [5]:
weather_data.columns = ['Branch_code', 'Branch_name', 'Date_time', 'Temperature', 'Wind_Speed(m/s)', 'Precipitation(mm)', 'Pressure(hPa)', 'Humidity(%)']

In [6]:
weather_data['Date_time'] = pd.to_datetime(weather_data['Date_time']).dt.strftime('%Y-%m-%d %H')
weather_data['Date_time_convert'] = pd.to_datetime(weather_data['Date_time'], infer_datetime_format = True)

In [7]:
weather_data['Year'] = weather_data['Date_time_convert'].dt.year
weather_data['Month'] = weather_data['Date_time_convert'].dt.month
weather_data['Date'] = weather_data['Date_time_convert'].dt.date
weather_data['Hour'] = weather_data['Date_time_convert'].dt.hour

weather_data.head()

,Branch_code,Branch_name,Date_time,Temperature,Wind_Speed(m/s),Precipitation(mm),Pressure(hPa),Humidity(%),Date_time_convert,Year,Month,Date,Hour
0,12,안면도(감),2022-01-01 00,-4.5,2.6,0.0,1026.1,58.9,2022-01-01 00:00:00,2022,1,2022-01-01,0
1,12,안면도(감),2022-01-01 01,-4.6,2.4,0.0,1026.4,57.7,2022-01-01 01:00:00,2022,1,2022-01-01,1
2,12,안면도(감),2022-01-01 02,-5.0,2.7,0.0,1026.0,65.2,2022-01-01 02:00:00,2022,1,2022-01-01,2
3,12,안면도(감),2022-01-01 03,-5.0,2.9,0.0,1026.0,54.2,2022-01-01 03:00:00,2022,1,2022-01-01,3
4,12,안면도(감),2022-01-01 04,-4.9,2.7,0.0,1026.1,51.2,2022-01-01 04:00:00,2022,1,2022-01-01,4


In [8]:
# Dedup the data to handle the repeating values on branch datetime level

weather_data = weather_data.drop_duplicates(subset=['Branch_name', 'Date_time'])

weather_data.shape

(6831630, 13)

In [9]:
null_variables = round((weather_data.isna().sum()/len(weather_data))*100,2)
null_variables

Branch_code           0.00
Branch_name           0.00
Date_time             0.00
Temperature           0.37
Wind_Speed(m/s)       0.34
Precipitation(mm)     1.13
Pressure(hPa)        48.07
Humidity(%)           9.65
Date_time_convert     0.00
Year                  0.00
Month                 0.00
Date                  0.00
Hour                  0.00
dtype: float64

In [10]:
#dropping the pressure variable as it has a greater % of nulls

weather_data = weather_data.drop(['Pressure(hPa)', 'Date_time_convert'], axis = 1)

In [11]:
### We will craete a subset dataset so that we can get the max temp and humidity for the day and along with that we can get the increase/descrease in next day readings
weather_data_subset = weather_data[['Branch_name', 'Date_time','Year', 'Month', 'Hour', 'Date', 'Temperature', 'Humidity(%)']]
weather_data_subset.head()

,Branch_name,Date_time,Year,Month,Hour,Date,Temperature,Humidity(%)
0,안면도(감),2022-01-01 00,2022,1,0,2022-01-01,-4.5,58.9
1,안면도(감),2022-01-01 01,2022,1,1,2022-01-01,-4.6,57.7
2,안면도(감),2022-01-01 02,2022,1,2,2022-01-01,-5.0,65.2
3,안면도(감),2022-01-01 03,2022,1,3,2022-01-01,-5.0,54.2
4,안면도(감),2022-01-01 04,2022,1,4,2022-01-01,-4.9,51.2


In [12]:
weather_data_subset['Temp_max_month'] = weather_data_subset.groupby(['Branch_name','Month'])['Temperature'].transform('max')
weather_data_subset['Temp_min_month'] = weather_data_subset.groupby(['Branch_name','Month'])['Temperature'].transform('min')
weather_data_subset['Temp_avg_month'] = weather_data_subset.groupby(['Branch_name','Month'])['Temperature'].transform('mean')

weather_data_subset['Humidity_max_month'] = weather_data_subset.groupby(['Branch_name','Month'])['Humidity(%)'].transform('max')
weather_data_subset['Humidity_min_month'] = weather_data_subset.groupby(['Branch_name','Month'])['Humidity(%)'].transform('min')
weather_data_subset['Humidity_avg_month'] = weather_data_subset.groupby(['Branch_name','Month'])['Humidity(%)'].transform('mean')

weather_data_subset['Temp_max_day'] = weather_data_subset.groupby(['Branch_name','Date'])['Temperature'].transform('max')
weather_data_subset['Temp_min_day'] = weather_data_subset.groupby(['Branch_name','Date',])['Temperature'].transform('min')
weather_data_subset['Temp_avg_day'] = weather_data_subset.groupby(['Branch_name','Date'])['Temperature'].transform('mean')

weather_data_subset['Humidity_max_day'] = weather_data_subset.groupby(['Branch_name','Date'])['Humidity(%)'].transform('max')
weather_data_subset['Humidity_min_day'] = weather_data_subset.groupby(['Branch_name','Date'])['Humidity(%)'].transform('min')
weather_data_subset['Humidity_avg_day'] = weather_data_subset.groupby(['Branch_name','Date'])['Humidity(%)'].transform('mean')

/tmp/ipykernel_44706/360319413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data_subset['Temp_max_month'] = weather_data_subset.groupby(['Branch_name','Month'])['Temperature'].transform('max')
/tmp/ipykernel_44706/360319413.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data_subset['Temp_min_month'] = weather_data_subset.groupby(['Branch_name','Month'])['Temperature'].transform('min')


In [13]:
weather_subset_month = weather_data_subset.copy()

weather_subset_month = weather_subset_month.drop_duplicates(subset=['Branch_name','Year', 'Month'])
weather_subset_month = weather_subset_month.sort_values(by = ['Branch_name','Year', 'Month'])

weather_subset_month.shape

(9411, 20)

In [14]:
weather_subset_month.head()

,Branch_name,Date_time,Year,Month,Hour,Date,Temperature,Humidity(%),Temp_max_month,Temp_min_month,Temp_avg_month,Humidity_max_month,Humidity_min_month,Humidity_avg_month,Temp_max_day,Temp_min_day,Temp_avg_day,Humidity_max_day,Humidity_min_day,Humidity_avg_day
21598,가거도,2021-01-04 21,2021,1,21,2021-01-04,6.2,49.9,15.6,-4.3,4.872472,99.7,35.7,67.284929,7.4,6.2,6.733333,65.1,49.9,58.800000
22249,가거도,2021-02-01 00,2021,2,0,2021-02-01,11.9,94.1,15.8,-2.8,5.272992,97.5,9.7,64.415625,11.9,6.1,9.770833,96.0,74.6,84.675000
22921,가거도,2021-03-01 00,2021,3,0,2021-03-01,13.9,88.5,16.3,3.3,9.071803,100.0,16.3,74.449192,14.9,6.1,11.204167,99.3,87.4,95.362500
21814,가거도,2021-04-01 00,2021,4,0,2021-04-01,14.0,48.9,20.5,5.4,12.612742,100.0,16.0,72.060272,14.0,11.2,12.125000,86.2,48.9,72.137500
22493,가거도,2021-05-01 00,2021,5,0,2021-05-01,10.7,87.4,23.5,8.5,16.609098,99.9,18.8,75.225465,12.8,8.9,11.079167,98.2,59.6,79.095833


In [15]:
monthly_difference_df = weather_subset_month[['Branch_name','Year', 'Month', 'Temp_max_month', 'Temp_min_month', 'Temp_avg_month', 'Humidity_max_month', 'Humidity_min_month', 'Humidity_avg_month']]

for var in ['Temp_max_month', 'Temp_min_month', 'Temp_avg_month', 'Humidity_max_month', 'Humidity_min_month', 'Humidity_avg_month']:
    monthly_difference_df['diff_'+var] = monthly_difference_df.groupby(['Branch_name'])[var].diff()

/tmp/ipykernel_44706/4089038031.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_difference_df['diff_'+var] = monthly_difference_df.groupby(['Branch_name'])[var].diff()
/tmp/ipykernel_44706/4089038031.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_difference_df['diff_'+var] = monthly_difference_df.groupby(['Branch_name'])[var].diff()
/tmp/ipykernel_44706/4089038031.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [16]:
monthly_difference_df.head()

,Branch_name,Year,Month,Temp_max_month,Temp_min_month,Temp_avg_month,Humidity_max_month,Humidity_min_month,Humidity_avg_month,diff_Temp_max_month,diff_Temp_min_month,diff_Temp_avg_month,diff_Humidity_max_month,diff_Humidity_min_month,diff_Humidity_avg_month
21598,가거도,2021,1,15.6,-4.3,4.872472,99.7,35.7,67.284929,NaN,NaN,NaN,NaN,NaN,NaN
22249,가거도,2021,2,15.8,-2.8,5.272992,97.5,9.7,64.415625,0.2,1.5,0.400519,-2.2,-26.0,-2.869304
22921,가거도,2021,3,16.3,3.3,9.071803,100.0,16.3,74.449192,0.5,6.1,3.798812,2.5,6.6,10.033567
21814,가거도,2021,4,20.5,5.4,12.612742,100.0,16.0,72.060272,4.2,2.1,3.540938,0.0,-0.3,-2.388920
22493,가거도,2021,5,23.5,8.5,16.609098,99.9,18.8,75.225465,3.0,3.1,3.996356,-0.1,2.8,3.165193


In [17]:
weather_subset_date = weather_data_subset.copy()

weather_subset_date = weather_subset_date.drop_duplicates(subset=['Branch_name','Date'])
weather_subset_date = weather_subset_date.sort_values(by = ['Branch_name','Date'])

weather_subset_date.shape

(284986, 20)

In [18]:
daily_difference_df = weather_subset_date[['Branch_name','Date', 'Temp_max_day', 'Temp_min_day', 'Temp_avg_day', 'Humidity_max_day', 'Humidity_min_day', 'Humidity_avg_day']]

for var in ['Temp_max_day', 'Temp_min_day', 'Temp_avg_day', 'Humidity_max_day', 'Humidity_min_day', 'Humidity_avg_day']:
    daily_difference_df['diff_'+var] = daily_difference_df.groupby(['Branch_name'])[var].diff()

/tmp/ipykernel_44706/1266898606.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_difference_df['diff_'+var] = daily_difference_df.groupby(['Branch_name'])[var].diff()
/tmp/ipykernel_44706/1266898606.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_difference_df['diff_'+var] = daily_difference_df.groupby(['Branch_name'])[var].diff()
/tmp/ipykernel_44706/1266898606.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [19]:
daily_difference_df.head(20)

,Branch_name,Date,Temp_max_day,Temp_min_day,Temp_avg_day,Humidity_max_day,Humidity_min_day,Humidity_avg_day,diff_Temp_max_day,diff_Temp_min_day,diff_Temp_avg_day,diff_Humidity_max_day,diff_Humidity_min_day,diff_Humidity_avg_day
21598,가거도,2021-01-04,7.4,6.2,6.733333,65.1,49.9,58.800000,NaN,NaN,NaN,NaN,NaN,NaN
21601,가거도,2021-01-05,7.8,2.0,4.695833,75.7,62.5,70.000000,0.4,-4.2,-2.037500,10.6,12.6,11.200000
21625,가거도,2021-01-06,3.5,0.8,2.208333,NaN,NaN,NaN,-4.3,-1.2,-2.487500,NaN,NaN,NaN
21649,가거도,2021-01-07,-0.1,-4.0,-2.766667,NaN,NaN,NaN,-3.6,-4.8,-4.975000,NaN,NaN,NaN
21673,가거도,2021-01-08,-1.2,-4.3,-2.704167,NaN,NaN,NaN,-1.1,-0.3,0.062500,NaN,NaN,NaN
21697,가거도,2021-01-09,1.3,-2.6,-0.287500,NaN,NaN,NaN,2.5,1.7,2.416667,NaN,NaN,NaN
21721,가거도,2021-01-10,3.7,1.1,2.550000,NaN,NaN,NaN,2.4,3.7,2.837500,NaN,NaN,NaN
21745,가거도,2021-01-11,3.5,0.4,2.054167,96.5,75.4,90.800000,-0.2,-0.7,-0.495833,NaN,NaN,NaN
21769,가거도,2021-01-12,6.4,2.2,3.712500,83.8,45.6,58.029167,2.9,1.8,1.658333,-12.7,-29.8,-32.770833
21793,가거도,2021-01-13,12.1,6.7,9.358333,67.2,40.0,53.862500,5.7,4.5,5.645833,-16.6,-5.6,-4.166667


In [20]:
### Joining the difference back into weather data so that it can be used later
weather_data_nonth_merge = pd.merge(weather_data, monthly_difference_df, how = 'left', left_on = ['Branch_name', 'Year', 'Month'], right_on = ['Branch_name', 'Year', 'Month'])

In [21]:
print(weather_data_nonth_merge.shape)
weather_data_nonth_merge.head()

(6831630, 23)


,Branch_code,Branch_name,Date_time,Temperature,Wind_Speed(m/s),Precipitation(mm),Humidity(%),Year,Month,Date,Hour,Temp_max_month,Temp_min_month,Temp_avg_month,Humidity_max_month,Humidity_min_month,Humidity_avg_month,diff_Temp_max_month,diff_Temp_min_month,diff_Temp_avg_month,diff_Humidity_max_month,diff_Humidity_min_month,diff_Humidity_avg_month
0,12,안면도(감),2022-01-01 00,-4.5,2.6,0.0,58.9,2022,1,2022-01-01,0,12.2,-13.0,0.174933,98.8,20.3,62.976075,-0.4,-4.6,-3.847379,4.7,2.9,0.485349
1,12,안면도(감),2022-01-01 01,-4.6,2.4,0.0,57.7,2022,1,2022-01-01,1,12.2,-13.0,0.174933,98.8,20.3,62.976075,-0.4,-4.6,-3.847379,4.7,2.9,0.485349
2,12,안면도(감),2022-01-01 02,-5.0,2.7,0.0,65.2,2022,1,2022-01-01,2,12.2,-13.0,0.174933,98.8,20.3,62.976075,-0.4,-4.6,-3.847379,4.7,2.9,0.485349
3,12,안면도(감),2022-01-01 03,-5.0,2.9,0.0,54.2,2022,1,2022-01-01,3,12.2,-13.0,0.174933,98.8,20.3,62.976075,-0.4,-4.6,-3.847379,4.7,2.9,0.485349
4,12,안면도(감),2022-01-01 04,-4.9,2.7,0.0,51.2,2022,1,2022-01-01,4,12.2,-13.0,0.174933,98.8,20.3,62.976075,-0.4,-4.6,-3.847379,4.7,2.9,0.485349


In [22]:
### Joining the difference back into weather data so that it can be used later
weather_data_final = pd.merge(weather_data_nonth_merge, daily_difference_df, how = 'left', left_on = ['Branch_name', 'Date'], right_on = ['Branch_name', 'Date'])

In [23]:
print(weather_data_final.shape)
weather_data_final.head()

(6831630, 35)


,Branch_code,Branch_name,Date_time,Temperature,Wind_Speed(m/s),Precipitation(mm),Humidity(%),Year,Month,Date,Hour,Temp_max_month,Temp_min_month,Temp_avg_month,Humidity_max_month,Humidity_min_month,Humidity_avg_month,diff_Temp_max_month,diff_Temp_min_month,diff_Temp_avg_month,diff_Humidity_max_month,diff_Humidity_min_month,diff_Humidity_avg_month,Temp_max_day,Temp_min_day,Temp_avg_day,Humidity_max_day,Humidity_min_day,Humidity_avg_day,diff_Temp_max_day,diff_Temp_min_day,diff_Temp_avg_day,diff_Humidity_max_day,diff_Humidity_min_day,diff_Humidity_avg_day
0,12,안면도(감),2022-01-01 00,-4.5,2.6,0.0,58.9,2022,1,2022-01-01,0,12.2,-13.0,0.174933,98.8,20.3,62.976075,-0.4,-4.6,-3.847379,4.7,2.9,0.485349,0.8,-5.6,-2.3625,65.2,36.3,51.208333,2.7,-1.5,0.704167,4.9,-9.0,-3.216667
1,12,안면도(감),2022-01-01 01,-4.6,2.4,0.0,57.7,2022,1,2022-01-01,1,12.2,-13.0,0.174933,98.8,20.3,62.976075,-0.4,-4.6,-3.847379,4.7,2.9,0.485349,0.8,-5.6,-2.3625,65.2,36.3,51.208333,2.7,-1.5,0.704167,4.9,-9.0,-3.216667
2,12,안면도(감),2022-01-01 02,-5.0,2.7,0.0,65.2,2022,1,2022-01-01,2,12.2,-13.0,0.174933,98.8,20.3,62.976075,-0.4,-4.6,-3.847379,4.7,2.9,0.485349,0.8,-5.6,-2.3625,65.2,36.3,51.208333,2.7,-1.5,0.704167,4.9,-9.0,-3.216667
3,12,안면도(감),2022-01-01 03,-5.0,2.9,0.0,54.2,2022,1,2022-01-01,3,12.2,-13.0,0.174933,98.8,20.3,62.976075,-0.4,-4.6,-3.847379,4.7,2.9,0.485349,0.8,-5.6,-2.3625,65.2,36.3,51.208333,2.7,-1.5,0.704167,4.9,-9.0,-3.216667
4,12,안면도(감),2022-01-01 04,-4.9,2.7,0.0,51.2,2022,1,2022-01-01,4,12.2,-13.0,0.174933,98.8,20.3,62.976075,-0.4,-4.6,-3.847379,4.7,2.9,0.485349,0.8,-5.6,-2.3625,65.2,36.3,51.208333,2.7,-1.5,0.704167,4.9,-9.0,-3.216667


In [24]:
#weather_data_final.to_csv('weather_data_2021_22_cleaned.csv', index = False , encoding ='euc_kr')

In [25]:
#!pip install fastparquet
#!pip install pyarrow

In [26]:
weather_data_final.to_parquet('weather_data_2021_22_cleaned.parquet')

In [27]:
new_df = pd.read_parquet('weather_data_2021_22_cleaned.parquet', engine='fastparquet')

In [28]:
new_df.shape

(6831630, 35)

In [29]:
new_df.head()

,Branch_code,Branch_name,Date_time,Temperature,Wind_Speed(m/s),Precipitation(mm),Humidity(%),Year,Month,Date,Hour,Temp_max_month,Temp_min_month,Temp_avg_month,Humidity_max_month,Humidity_min_month,Humidity_avg_month,diff_Temp_max_month,diff_Temp_min_month,diff_Temp_avg_month,diff_Humidity_max_month,diff_Humidity_min_month,diff_Humidity_avg_month,Temp_max_day,Temp_min_day,Temp_avg_day,Humidity_max_day,Humidity_min_day,Humidity_avg_day,diff_Temp_max_day,diff_Temp_min_day,diff_Temp_avg_day,diff_Humidity_max_day,diff_Humidity_min_day,diff_Humidity_avg_day
0,12,안면도(감),2022-01-01 00,-4.5,2.6,0.0,58.9,2022,1,2022-01-01,0,12.2,-13.0,0.174933,98.8,20.3,62.976075,-0.4,-4.6,-3.847379,4.7,2.9,0.485349,0.8,-5.6,-2.3625,65.2,36.3,51.208333,2.7,-1.5,0.704167,4.9,-9.0,-3.216667
1,12,안면도(감),2022-01-01 01,-4.6,2.4,0.0,57.7,2022,1,2022-01-01,1,12.2,-13.0,0.174933,98.8,20.3,62.976075,-0.4,-4.6,-3.847379,4.7,2.9,0.485349,0.8,-5.6,-2.3625,65.2,36.3,51.208333,2.7,-1.5,0.704167,4.9,-9.0,-3.216667
2,12,안면도(감),2022-01-01 02,-5.0,2.7,0.0,65.2,2022,1,2022-01-01,2,12.2,-13.0,0.174933,98.8,20.3,62.976075,-0.4,-4.6,-3.847379,4.7,2.9,0.485349,0.8,-5.6,-2.3625,65.2,36.3,51.208333,2.7,-1.5,0.704167,4.9,-9.0,-3.216667
3,12,안면도(감),2022-01-01 03,-5.0,2.9,0.0,54.2,2022,1,2022-01-01,3,12.2,-13.0,0.174933,98.8,20.3,62.976075,-0.4,-4.6,-3.847379,4.7,2.9,0.485349,0.8,-5.6,-2.3625,65.2,36.3,51.208333,2.7,-1.5,0.704167,4.9,-9.0,-3.216667
4,12,안면도(감),2022-01-01 04,-4.9,2.7,0.0,51.2,2022,1,2022-01-01,4,12.2,-13.0,0.174933,98.8,20.3,62.976075,-0.4,-4.6,-3.847379,4.7,2.9,0.485349,0.8,-5.6,-2.3625,65.2,36.3,51.208333,2.7,-1.5,0.704167,4.9,-9.0,-3.216667
